In [9]:
import pandas as pd
import sqlalchemy as sa
import pywikidata as wp

In [10]:
# Read a sample dataset of 44107 cities from wikidata with their population
wikidata_query = """
SELECT ?city ?cityLabel ?population
WHERE {
  ?city wdt:P31/wdt:P279* wd:Q515.
  OPTIONAL { ?city wdt:P1082 ?population. }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
"""

# read predownloaded dataset. This is a sample of 44107 cities from wikidata with their population
df_wiki = pd.read_csv('sample_data/query_cities_population_entityid.csv')

# Sort dataframe in ascending order by population
df_wiki.sort_values(by=['population'], ascending=True, inplace=True)
df_wiki.head()

,city,cityLabel,population
41018,http://www.wikidata.org/entity/Q20541864,Lousadela,0
15985,http://www.wikidata.org/entity/Q48791,Tower City,0
40007,http://www.wikidata.org/entity/Q12381764,Q12381764,0
33214,http://www.wikidata.org/entity/Q1991598,South Park View,0
4335,http://www.wikidata.org/entity/Q1475379,Rineia,0


In [11]:
# MusicBrainz database credentials
HOST = "localhost"
DATABASE = "musicbrainz_db"
USER = "musicbrainz"
PORT = 5432

sa_conn_str = f"postgresql+psycopg2://{USER}@{HOST}:{PORT}/{DATABASE}"
engine = sa.create_engine(sa_conn_str)

In [12]:
with engine.connect() as conn:
    query = '''
    SELECT a.name as name, at.name as type
    FROM area AS a 
    LEFT JOIN area_type as at 
    ON a.type = at.id;
    '''
    
    df_musicbrainz = pd.read_sql(sa.text(query), conn)

# Fetches all ~120k areas from the MusicBrainz database with their area type
df_musicbrainz

,name,type
0,Greccio,Municipality
1,Canada,Country
2,Chile,Country
3,China,Country
4,Cambodia,Country
...,...,...
118959,Fairview Beach,City
118960,Sherman Oaks,District
118961,Islay,Island
118962,Bowmore,City


In [13]:
# Convert all names to lowercase for comparision
df_musicbrainz['name'] = df_musicbrainz['name'].str.lower()
df_wiki['cityLabel'] = df_wiki['cityLabel'].str.lower()

In [14]:
not_in_MeB = df_wiki[~df_wiki.cityLabel.isin(df_musicbrainz.name)]
is_in_MeB = df_wiki[df_wiki.cityLabel.isin(df_musicbrainz.name)]

In [21]:
# Previewing cities that are NOT in MusicBrainz database
not_in_MeB.sort_values(by = 'population', ascending=False)

,city,cityLabel,population
5456,http://www.wikidata.org/entity/Q1020110,mumias,99987
3152,http://www.wikidata.org/entity/Q740499,bishoftu,99928
2247,http://www.wikidata.org/entity/Q405663,pavlikeni,9986
7765,http://www.wikidata.org/entity/Q2535326,adășeni,998
15190,http://www.wikidata.org/entity/Q155281,kuldīga,9974
...,...,...,...
34930,http://www.wikidata.org/entity/Q6026356,mundobriga,0
40207,http://www.wikidata.org/entity/Q12393950,q12393950,0
44040,http://www.wikidata.org/entity/Q61379203,q61379203,0
41130,http://www.wikidata.org/entity/Q113513507,a garduñeira,0


In [18]:
# Only previewing cities with population > 1000, 
# as cities with population < 1000 are most likely fake in our case.
not_in_MeB[not_in_MeB.population.astype(float) > 1000]

,city,cityLabel,population
3146,http://www.wikidata.org/entity/Q739353,dorestad,10000
32603,http://www.wikidata.org/entity/Q739353,dorestad,10000
9011,http://www.wikidata.org/entity/Q2604911,bidjabidjan,10000
2501,http://www.wikidata.org/entity/Q615707,"pedernales, delta amacuro",10000
6778,http://www.wikidata.org/entity/Q1404020,ras gharib,100000
...,...,...,...
34057,http://www.wikidata.org/entity/Q155281,kuldīga,9974
15190,http://www.wikidata.org/entity/Q155281,kuldīga,9974
2247,http://www.wikidata.org/entity/Q405663,pavlikeni,9986
3152,http://www.wikidata.org/entity/Q740499,bishoftu,99928
